In [1]:
import pandas as pd

In [21]:
import preprocess
data =pd.read_csv('assets/dataset.csv', delimiter=";", encoding='latin-1')

# pre process the data
# my_df['Year'] = my_df['Year'].astype('int')


# list(my_df.columns)
formes_juridiques = data["Form_juridique"].dropna().unique()

years = list(data["Year"].dropna().unique())
years = list(map(lambda x: int(x), years))
years.sort()

modes_transmission = list(data["Mode_transmission"].dropna().unique())
modes_transmission = list(map(lambda s: s.capitalize(), modes_transmission))


Particuliers = len(
    data[data["Form_juridique"] == "P"].groupby("Num_contribuable").count()
)
Entreprises = len(
    data[data["Form_juridique"] == "C"].groupby("Num_contribuable").count()
)
Syndicats = len(data[data["Form_juridique"] == "S"].groupby("Num_contribuable").count())

Effectif = len(data["Num_contribuable"].unique())
Timeline = (
    "Données recensées de "
    + str(int(data["Year"].dropna().unique()[0]))
    + " à "
    + str(int(data["Year"].dropna().unique()[-1]))
)
regions = list(data["Region"].dropna().unique())
regions = list(map(lambda s: s.capitalize(), regions))
Nb_regions = len(regions)

print("Lecture fichier ok")

data_IC = data[data["Form_juridique"] == "C"]
data_IP = data[data["Form_juridique"] == "P"]
data_S = data[data["Form_juridique"] == "S"]

data_IC = preprocess.clean_names(data_IC)
data_IC = preprocess.remove_missing_values(data_IC, "IC")
data_IC = preprocess.convert_types(data_IC, "IC")
data_IC = preprocess.sort_by_yr(data_IC)

data_IP = preprocess.clean_names(data_IP)
data_IP = preprocess.remove_missing_values(data_IP, "IP")
data_IP = preprocess.convert_types(data_IP, "IP")
data_IP = preprocess.sort_by_yr(data_IP)

data_S = preprocess.clean_names(data_S)
data_S = preprocess.remove_missing_values(data_S, None)
data_S = preprocess.convert_types(data_S, None)
data_S = preprocess.sort_by_yr(data_S)

data_whole = preprocess.combine_dfs(data_IC, data_IP, data_S)

data_whole

Lecture fichier ok
Lignes avant la suppression des valeurs manquantes :  120570
Lignes après la suppression des valeurs manquantes :  82543
Lignes avant la suppression des valeurs manquantes :  66525
Lignes après la suppression des valeurs manquantes :  7207
Lignes avant la suppression des valeurs manquantes :  4265
Lignes après la suppression des valeurs manquantes :  1230


,Num_contribuable,Year,Taille,Form_juridique,Region,Produire_IP,Produire_IC,Produire_TVQ,Produire_RAS,Declarer_IP,Declarer_IC,Declarer_TVQ,Declarer_RAS,Mode_transmission
0,1,2014,Moyenne,C,Montreal,<NA>,1,1,0,<NA>,0,1,1,Logiciel
1,3835,2014,Petite,C,Lanaudiere,<NA>,0,0,0,<NA>,1,1,0,Logiciel
2,29888,2014,Petite,C,Laurentides,<NA>,0,0,0,<NA>,0,1,0,Logiciel
3,26691,2014,Petite,C,Monteregie,<NA>,0,0,0,<NA>,1,1,1,Logiciel
4,3832,2014,Petite,C,Lanaudiere,<NA>,0,1,1,<NA>,0,0,1,Logiciel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90975,27705,2018,Petite,S,Monteregie,<NA>,<NA>,0,1,<NA>,<NA>,1,1,Indeterminé
90976,6800,2018,Petite,S,Lanaudiere,<NA>,<NA>,1,1,<NA>,<NA>,0,1,Indeterminé
90977,22070,2018,Petite,S,Montreal,<NA>,<NA>,1,1,<NA>,<NA>,1,1,Indeterminé
90978,17295,2018,Petite,S,Lanaudiere,<NA>,<NA>,0,0,<NA>,<NA>,0,0,Indeterminé


In [33]:
def filter_line_chart_df(my_df,region,obligation,indicateur,transmission):
   my_df = my_df[(my_df['Region'] == region) 
                     & (my_df['Mode_transmission'] == transmission)]
   my_df = my_df.groupby(["Year","Form_juridique"]).agg({'Produire_IP': 'mean', 'Declarer_IP': 'mean',
                                                    'Produire_IC': 'mean', 'Declarer_IC': 'mean',
                                                    'Produire_RAS': 'mean', 'Declarer_RAS': 'mean',
                                                    'Produire_TVQ': 'mean', 'Declarer_TVQ': 'mean'}).reset_index()
   if obligation == 'Declarer':
      column_to_select = "Declarer_" + indicateur
   elif obligation == 'Produire':
      column_to_select = "Produire_" + indicateur
   else:
      raise(Exception)
   my_df = my_df[["Year", "Form_juridique", column_to_select]]
   my_df = my_df.rename(columns={column_to_select: "Valeurs"})
   return my_df

filter_line_chart_df(data_whole,"Montreal","Declarer","TVQ","Logiciel")

,Year,Form_juridique,Valeurs
0,2014,C,0.495392
1,2014,P,0.49505
2,2015,C,0.510816
3,2015,P,0.514286
4,2016,C,0.497758
5,2016,P,0.473214
6,2017,C,0.489387
7,2017,P,0.422414
8,2018,C,0.486171
9,2018,P,0.527778
